In [11]:
# Input: Boss stats

boss_points: 55
boss_damage: 8

# Player stats:
player_points = 50 
player_mana = 500

In [8]:
# Magic Missile costs 53 mana. It instantly does 4 damage.
# Drain costs 73 mana. It instantly does 2 damage and heals you for 2 hit points.
# Shield costs 113 mana. It starts an effect that lasts for 6 turns. While it is active, your armor is increased by 7.
# Poison costs 173 mana. It starts an effect that lasts for 6 turns. At the start of each turn while it is active, it deals the boss 3 damage.
# Recharge costs 229 mana. It starts an effect that lasts for 5 turns. At the start of each turn while it is active, it gives you 101 new mana.





class Game:
    def __init__(this, player, boss, verbose=True):
        this.player = player
        this.boss = boss
        this.game_over = False
        this.spells_history = []
        this.verbose = verbose
        this.bufs = {"poison":[], "armor":[], "mana":[]}
        this.total_mana = 0
        # g.print_stats()


    def print(this, *args):
        if this.verbose:
            print(*args)


    def use_mana(this, amount):
        if this.player['mana'] >= amount:
            this.player['mana'] -= amount
            this.total_mana += amount
            return True
        else:
            print('Player is out of mana!')
            return False
    

    def apply_poison_damage(this):
        if len(this.bufs['poison']) > 0:
            dmg = this.bufs['poison'].pop()
            this.print(f'Poison deals {dmg} damage,  remaining poison', this.bufs['poison'])
            this.boss['points'] -= dmg

            if this.boss['points'] <= 0:
                this.print('Boss is dead!')
                this.game_over = True


    def apply_recharge(this):
         if len(this.bufs['mana']) > 0:
            
            this.player['mana'] += this.bufs['mana'].pop()
            this.print(f'Recharge provides 101 mana; its timer is now', this.bufs['mana'])

    def apply_armor(this):
        if len(this.bufs['armor']) > 0:
            armor = this.bufs['armor'].pop()  
            if len(this.bufs['armor']) == 0:
                this.print('Armor wears off!')
            
        else:
            armor = 0

        return armor


    def player_move(this, spell):
        """
        Player move
        """
        if this.game_over:
            return 
        this.print('-- Player turn --')
        this.print(f"- Player has {this.player['points']} hitpoints, {this.bufs['armor']} armor, {this.player['mana']} mana")
        this.print(f"- Boss has {this.boss['points']} hit points")

        if this.player.get('mode', 'easy') == 'hard':
            this.apply_player_damage(1)

        this.apply_poison_damage()
        this.print(f'Player casts {spell}')

        # todo.. use mana here..

        # if len(this.bufs['mana']) > 0:
        #     print('Player gets new mana!')
        #     this.player['mana'] += this.bufs['mana'].pop()

        this.apply_recharge()

        this.apply_armor()
        
        if spell == 'Magic Missile':
            # Magic Missile costs 53 mana. It instantly does 4 damage.
            if this.use_mana(53):
                this.boss['points'] -= 4
            else:
                this.print('Game Over')
                this.game_over = True
                return

        elif spell == 'Drain':
            # Drain costs 73 mana. It instantly does 2 damage and heals you for 2 hit points.
            if this.use_mana(73):
                this.boss['points'] -= 2
                this.player['points'] += 2
            else:
                print('Game Over')
                this.game_over = True
                return

        elif spell == 'Shield':
            # Shield costs 113 mana. 
            # It starts an effect that lasts for 6 turns. While it is active, your armor is increased by 7.
            if this.use_mana(113):
                this.bufs['armor'] = this.bufs['armor'] + ([7] * 6)
            else:
                this.print('Game Over')
                this.game_over = True
                return

        elif spell == 'Poison':
            # poison takes effect at boss moves..
            # Poison costs 173 mana. It starts an effect that lasts for 6 turns. 
            # At the start of each turn while it is active, it deals the boss 3 damage.
            if this.use_mana(173):
                this.bufs['poison'] = this.bufs['poison'] + ([3] * 6)
            else:
                this.print('Game Over')
                this.game_over = True
                return
            
        elif spell == 'Recharge':
            # Recharge costs 229 mana. 
            # It starts an effect that lasts for 5 turns. At the start of each turn while it is active, it gives you 101 new mana.
            if this.use_mana(229):
                this.bufs['mana'] =  this.bufs['mana'] + ([101] * 5)
                this.apply_recharge()
            else:
                this.print('Game Over')
                this.game_over = True

        else:
            print('Wrong spell!')


        this.print('')
    
    
    def apply_player_damage(this, attack):

        this.player['points'] -= attack
        this.print(f'Player receives {attack} damage')
        if this.player['points'] <= 0:
            this.print(' ---- Game over ----')
            this.game_over = True
            return


    def boss_move(this):
        """
        Boss move
        """
        if this.game_over:
            return 
        
        this.print("-- Boss turn --")
        this.print(f"- Player has {this.player['points']} health", this.bufs['armor'], 'armor. ', this.player['mana'],' mana')
        this.print(f"- Boss has {this.boss['points']} hit points")

        # if len(this.bufs['poison']) > 0:
        #     dmg = this.bufs['poison'].pop()
        #     print(f'Poison deals {dmg} damage,  remaining poison', this.bufs['poison'])
        #     this.boss['points'] -= dmg
        this.apply_poison_damage()
        this.apply_recharge()


        
        armor = this.apply_armor()
        

        attack = max(1, this.boss['damage'] - armor )

        this.apply_player_damage(attack)
        # this.player['points'] -= attack
        # if this.player['points'] <= 0:
        #     this.print('Game over')
        #     this.game_over = True
        #     return

        this.print(f'Player uses {armor} armor, boss attacks for {attack} damage')

        this.print('')
    
    def turn(this, spell):
        # player moves first

        this.spells_history.append(spell)
        this.player_move(spell)

        this.boss_move()

    

    def result(this):
        if this.player['mana'] <= 0:
            
            return 'Player ran out of mana!', 1
        elif this.boss['points'] <= 0:
            return 'Player won', 2
        
        elif this.player['points'] <= 0:
            return 'Boss won!', 3
    
        else:
            # print('The game is on')
            return 'Game is on!', 0



    def player_won(this):
        return this.boss['points'] <= 0
    
    def available_spells(this):
        spells = ['Poison','Magic Missile', 'Shield', 'Drain','Recharge']
        result = []

       
        

        mana = this.player['mana']
        if mana >= 53:
            result.append('Magic Missile')
        if mana >= 73:
            result.append('Drain')



        if (mana >= 173) and (len(this.bufs['poison'])==0):
            result.append('Poison') 

        if (mana >= 113) and (len(this.bufs['armor'])==0):
            result.append('Shield') 



        if (mana >= 229) and (len(this.bufs['mana'])==0):
            result.append('Recharge') 
        return result

# g.turn('Recharge')
# g.turn('Shield')
# g.turn('Drain')

# correct...
g = Game({'points':10, 'mana':250},{'points':13, 'damage':8}, verbose=True)

g.turn('Poison') #,      173,    3,      0,      0,        0,      6))
g.turn('Magic Missile')#, 53,    4,      0,      0,        0,      1))

g.game_over




-- Player turn --
- Player has 10 hitpoints, [] armor, 250 mana
- Boss has 13 hit points
Player casts Poison

-- Boss turn --
- Player has 10 health [] armor.  77  mana
- Boss has 13 hit points
Poison deals 3 damage,  remaining poison [3, 3, 3, 3, 3]
Player receives 8 damage
Player uses 0 armor, boss attacks for 8 damage

-- Player turn --
- Player has 2 hitpoints, [] armor, 77 mana
- Boss has 10 hit points
Poison deals 3 damage,  remaining poison [3, 3, 3, 3]
Player casts Magic Missile

-- Boss turn --
- Player has 2 health [] armor.  24  mana
- Boss has 3 hit points
Poison deals 3 damage,  remaining poison [3, 3, 3]
Boss is dead!
Player receives 8 damage
 ---- Game over ----
Player uses 0 armor, boss attacks for 8 damage



True

In [9]:
g = Game({'points':10, 'mana':250, 'mode':'hard'},{'points':14, 'damage':8})


# print(g.available_spells())
g.turn('Recharge')
# print(g.available_spells())
g.turn('Shield')
g.turn('Drain')
g.turn('Poison')
g.turn('Magic Missile')

-- Player turn --
- Player has 10 hitpoints, [] armor, 250 mana
- Boss has 14 hit points
Player receives 1 damage
Player casts Recharge
Recharge provides 101 mana; its timer is now [101, 101, 101, 101]

-- Boss turn --
- Player has 9 health [] armor.  122  mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now [101, 101, 101]
Player receives 8 damage
Player uses 0 armor, boss attacks for 8 damage

-- Player turn --
- Player has 1 hitpoints, [] armor, 223 mana
- Boss has 14 hit points
Player receives 1 damage
 ---- Game over ----
Player casts Shield
Recharge provides 101 mana; its timer is now [101, 101]



# Let us draw a graph..

In [3]:
from copy import deepcopy




def explore_game_depth_first(game):

    if len(game.spells_history) >= 10:
        print('Max depth reached! Killing recursion')
        return
    spells = game.available_spells()
    msg, r = game.result()
    if  r != 0:
        print(game.spells_history)
        print(msg)
    for spell in spells:
        # print('Casting ', spell)
        k = deepcopy(game)
        k.turn(spell)
        explore_game_depth_first(k)


def explore_game_breadth_first(game):
   
    min_mana_to_win = 999999999999
   
    spells = game.available_spells()
    buf = [(spell, deepcopy(game)) for spell in spells]
    n = 0
    while len(buf) > 0:
        # print([a[0] for a in buf])
        spell, g = buf.pop(0)
        g.turn(spell)
        msg, r = g.result()
        # print(g.spells_history, msg)
        if r == 0:
            spells = g.available_spells()
            if len(g.spells_history) < 10:
                
                buf = buf + [(spell, deepcopy(g)) for spell in spells]
        else:
            # if g.spells_history == ['Poison', 'Magic Missile']:
                # print('-- Kanyee --')
                # print(g.player, g.boss)
            if r ==2: # Player won
                print(g.spells_history, msg, g.total_mana)
                if g.total_mana < min_mana_to_win:
                    min_mana_to_win = g.total_mana

        n = n+1
    print('Answer part I = ', min_mana_to_win)


In [ ]:


g = Game({'points':50, 'mana':500},{'points':55, 'damage':8}, verbose=False)


# print(g.available_spells())
# explore_game_depth_first(g)
explore_game_breadth_first(g)


In [10]:
g = Game({'points':10, 'mana':250, 'mode':'hard'},{'points':13, 'damage':8}, verbose=True)

g.turn('Poison') #,      173,    3,      0,      0,        0,      6))
g.turn('Magic Missile')#, 53,    4,      0,      0,        0,      1))

g.game_over


-- Player turn --
- Player has 10 hitpoints, [] armor, 250 mana
- Boss has 13 hit points
Player receives 1 damage
Player casts Poison

-- Boss turn --
- Player has 9 health [] armor.  77  mana
- Boss has 13 hit points
Poison deals 3 damage,  remaining poison [3, 3, 3, 3, 3]
Player receives 8 damage
Player uses 0 armor, boss attacks for 8 damage

-- Player turn --
- Player has 1 hitpoints, [] armor, 77 mana
- Boss has 10 hit points
Player receives 1 damage
 ---- Game over ----
Poison deals 3 damage,  remaining poison [3, 3, 3, 3]
Player casts Magic Missile



True

# Part II

With little help from the internet.

I think my algorithm incorrectly calculates available spells, when they are about to run out..

In [14]:
g = Game({'points':50, 'mana':500, 'mode':'hard'},{'points':55, 'damage':8}, verbose=True)


spells =  ['Poison', 'Recharge', 'Shield', 'Poison', 'Recharge', 'Drain', 'Poison', 'Drain', 'Magic Missile']

for spell in spells:
    g.turn(spell)



# 
# g.turn('Magic Missile')
# g.turn('Recharge')
# explore_game_breadth_first(g)


# ['Poison', 'Magic Missile', 'Recharge', 'Shield', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Magic Missile'] Player won 1362
# ['Poison', 'Magic Missile', 'Recharge', 'Shield', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Drain'] Player won 1382
# ['Poison', 'Magic Missile', 'Recharge', 'Shield', 'Poison', 'Recharge', 'Drain', 'Shield', 'Poison', 'Magic Missile'] Player won 1382
# ['Poison', 'Drain', 'Recharge', 'Shield', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Magic Missile'] Player won 1382
# ['Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Magic Missile'] Player won 1362
# ['Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Drain'] Player won 1382
# ['Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Recharge', 'Drain', 'Shield', 'Poison', 'Magic Missile'] Player won 1382
# ['Poison', 'Recharge', 'Drain', 'Shield', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Magic Missile'] Player won 1382
# ['Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Magic Missile'] Player won 1362
# ['Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Drain'] Player won 1382
# ['Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Recharge', 'Drain', 'Shield', 'Poison', 'Magic Missile'] Player won 1382
# ['Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Magic Missile'] Player won 1362
# ['Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Drain'] Player won 1382
# ['Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Recharge', 'Shield', 'Drain', 'Poison', 'Magic Missile'] Player won 1382
# ['Poison', 'Recharge', 'Shield', 'Drain', 'Poison', 'Recharge', 'Magic Missile', 'Shield', 'Poison', 'Magic Missile'] Player won 1382
# ['Poison', 'Recharge', 'Shield', 'Drain', 'Poison', 'Recharge', 'Shield', 'Magic Missile', 'Poison', 'Magic Missile'] Player won 1382
# Answer part I =  1362



# 1362 -- too high..

-- Player turn --
- Player has 50 hitpoints, [] armor, 500 mana
- Boss has 55 hit points
Player receives 1 damage
Player casts Poison

-- Boss turn --
- Player has 49 health [] armor.  327  mana
- Boss has 55 hit points
Poison deals 3 damage,  remaining poison [3, 3, 3, 3, 3]
Player receives 8 damage
Player uses 0 armor, boss attacks for 8 damage

-- Player turn --
- Player has 41 hitpoints, [] armor, 327 mana
- Boss has 52 hit points
Player receives 1 damage
Poison deals 3 damage,  remaining poison [3, 3, 3, 3]
Player casts Recharge
Recharge provides 101 mana; its timer is now [101, 101, 101, 101]

-- Boss turn --
- Player has 40 health [] armor.  199  mana
- Boss has 49 hit points
Poison deals 3 damage,  remaining poison [3, 3, 3]
Recharge provides 101 mana; its timer is now [101, 101, 101]
Player receives 8 damage
Player uses 0 armor, boss attacks for 8 damage

-- Player turn --
- Player has 32 hitpoints, [] armor, 300 mana
- Boss has 46 hit points
Player receives 1 damage
Poison de

In [15]:
g.total_mana

1289